In [1]:
import npnet as tn
import ocr_data

import numpy as np

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,7]
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['ytick.right'] = True
matplotlib.rcParams['ytick.direction'] = 'in'
matplotlib.rcParams['ytick.minor.visible'] = True
matplotlib.rcParams['font.size'] = 19
matplotlib.rcParams['font.family'] = 'DejaVu Serif'
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams['errorbar.capsize'] = 3

In [3]:
input_shape = ocr_data.in_2d_shape
conv_kernels = [] #[(5,5)] #[(3,3),(3,3),(3,3),(3,3),(3,3)]
conv_stride = [] #[(3,3)] #[(1,1),(2,2),(1,1),(2,2),(1,1)]
conv_outs = [] #[(4,)] #[(4,),(8,),(8,),(16,),(16,)]
conv_layers = len(conv_kernels)
hidden_shapes = [(10,10)]
hidden_layers = len(hidden_shapes)
residual_layers = 4
more_hidden_shapes = []
more_hidden_layers = len(hidden_shapes)
output_shape = ocr_data.out_shape
print(input_shape,output_shape)

(32, 32) (26,)


In [4]:
in_layer = tn.Input(input_shape)()
last_layer = in_layer
print(in_layer)
for kernel_shape,out_shape,kernel_stride in zip(conv_kernels,conv_outs,conv_stride):
    last_layer = tn.Conv(kernel_shape,out_shape=out_shape,kernel_stride=kernel_stride,pad=True,activation=tn.ReLU())(last_layer)
    print(last_layer)
for hidden_shape in hidden_shapes:
    last_layer = tn.Dense(hidden_shape,activation=tn.ReLU())(last_layer)
    print(last_layer)
for i in range(residual_layers):
    last_layer = tn.ResWrap(tn.Dense(last_layer.output_shapes[0],activation=tn.ReLU()))(last_layer)
    print(last_layer)
for hidden_shape in more_hidden_shapes:
    last_layer = tn.Dense(hidden_shape,activation=tn.ReLU())(last_layer)
    print(last_layer)
out_layer = tn.Dense(output_shape,activation=tn.ReLU())(last_layer)
print(out_layer)

s = tn.System(inputs=[in_layer],outputs=[out_layer])

Input :: [] -> [(32, 32)]
Dense :: [(32, 32)] -> [(10, 10)]
ResWrap :: [(10, 10)] -> [(10, 10)]
ResWrap :: [(10, 10)] -> [(10, 10)]
ResWrap :: [(10, 10)] -> [(10, 10)]
ResWrap :: [(10, 10)] -> [(10, 10)]
Dense :: [(10, 10)] -> [(26,)]
ResWrap :: [(10, 10)] -> [(10, 10)] idx: 1 => Dense :: [(10, 10)] -> [(26,)] idx: 0
Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 2 => ResWrap :: [(10, 10)] -> [(10, 10)] idx: 1
ResWrap :: [(10, 10)] -> [(10, 10)] idx: 3 => Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 2
Dense :: [(10, 10)] -> [(10, 10)] idx: 4 => Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 2
Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 5 => ResWrap :: [(10, 10)] -> [(10, 10)] idx: 3
Dense :: [(10, 10)] -> [(10, 10)] idx: 6 => Dense :: [(10, 10)] -> [(10, 10)] idx: 4
ResWrap :: [(10, 10)] -> [(10, 10)] idx: 7 => Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 5
Dense :: [(10, 10)] -> [(10, 10)] idx: 8 => Add :: [(10, 10), (10, 10)] -> [(10, 10)] idx: 5
ResWrap :: [(10, 10)] -> [(10,

In [5]:
import multiprocessing
import functools

def batch(length=1000):    
    total,failures = 0,0
    state = None
    for true_out,input in ocr_data.tagged_2d_data(length):
        guess_out,state = s.guess([input],return_state=True)
        #print(guess_out[0]-true_out)
        if np.argmax(guess_out[0]) != np.argmax(true_out):
            failures += 1
        s.learn(state,[true_out],scale=1e-2,loss='quad')
        total += 1
    return total,failures

In [6]:
print('loading weights')
#s.load_weights('OCR_residual_network.h5')

loading weights


In [7]:
#%prun batch(100)

In [ ]:
try:
    batch_size = 10000
    while True:
        cases,failures = batch(batch_size)
        print('saving weights')
        s.save_weights('OCR_residual_network.h5')
        print('batch accuracy',cases-failures,'/',cases)
        if (cases-failures)/cases > 0.98:
            break
except KeyboardInterrupt:
    print('Stopped by user')

saving weights
batch accuracy 3490 / 10000
saving weights
batch accuracy 3515 / 10000
saving weights
batch accuracy 3557 / 10000
saving weights
batch accuracy 3641 / 10000
saving weights
batch accuracy 3599 / 10000
saving weights
batch accuracy 3770 / 10000
saving weights
batch accuracy 3737 / 10000
saving weights
batch accuracy 3680 / 10000
saving weights
batch accuracy 3839 / 10000
saving weights
batch accuracy 3902 / 10000
saving weights
batch accuracy 3807 / 10000
saving weights
batch accuracy 3705 / 10000
saving weights
batch accuracy 3977 / 10000
saving weights
batch accuracy 3840 / 10000
saving weights
batch accuracy 3855 / 10000
saving weights
batch accuracy 3989 / 10000
saving weights
batch accuracy 4055 / 10000
saving weights
batch accuracy 4114 / 10000
saving weights
batch accuracy 3788 / 10000
saving weights
batch accuracy 4054 / 10000
saving weights
batch accuracy 3920 / 10000
saving weights
batch accuracy 4099 / 10000
saving weights
batch accuracy 4106 / 10000
saving weig

In [ ]:
for i,c in enumerate(range(ord('A'),ord('Z'))):
    print('neuron',chr(c))
    plt.imshow(s.parts[0].layer[0].weights[i,:].reshape(hidden_shapes[-1]))
    plt.colorbar()
    plt.show()
    plt.close()

In [ ]:
for i in range(np.prod(hidden_shapes[0])):
    print('layer 0 neuron',i)
    plt.imshow(s.parts[15].layer[0].weights[i,:].reshape((32,32)))
    plt.colorbar()
    plt.show()
    plt.close()